# Course Project: What's this gene?

In this project you will implement a de-novo sequence assembly algorithm. You will be provided with a small sample of fragments (e.g. from an Illumina type machine) for a part of a well-known protein encoding human gene. Your task is to assemble the reads into a sequence then perform an online [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi) search to find out what the gene is.

This project will see you use all of the techniques you will learn along the way in the course and during the course will will provide you with opportunities to work on the project.

---

## Background

We will use a ["greedy algorithm"](https://en.wikipedia.org/wiki/Sequence_assembly#Greedy_algorithm) for sequence assembly. This is because it is straightforward to understand and implement but will also give you good enough results to solve the challenge. The steps of the algorithm are:

1. Сalculate pairwise alignments of all fragments.
1. Choose two fragments with the largest overlap.
1. Merge chosen fragments.
1. Repeat step 2 and 3 until only one fragment is left or you cannot merge anymore.

In the following code cell, we have supplied the fragments you should assemble the sequence from.

In [ ]:
fragments = [
    "GAATTAGATAAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATAAAAGTCTTCGCACAGTGAAAACTAAAATGGATCAAGCAGATGATG",
    "TTCTTCAGAAGCTCCACCCTATAATTCTGAACCTGCAGAAGAATCTGAACATAAAAACAACAATTACGAACCAAACCTATTTAAAACTCCACAAAGGAAA",
    "TGCCTATTGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGA",
    "CTCCACAAAGGAAACCATCTTATAATCAGCTGGCTTCAACTCCAATAATATTCAAAGAGCAAGGGCTGACTCTGCCGCTGTACCAATCTCCTGTAAAAGA",
    "CAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGAACTTTCTTCAGAAGCTCCACCCTA",
    "GGCTGACTCTGCCGCTGTACCAATCTCCTGTAAAAGAATTAGATAAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATAAAAGTCTTCG",
    "CAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGAACTTTCTTCAGAA",
    "AGCAAGGGCTGACTCTGCCGCTGTACCAATCTCCTGTAAAAGAATTAGATAAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATAAAAG",
    "TCAGCTGGCTTCAACTCCAATAATATTCAAAGAGCAAGGGCTGACTCTGCCGCTGTACCAATCTCCTGTAAAAGAATTAGATAAATTCAAATTAGACTTA",
    "AATCTCCTGTAAAAGAATTAGATAAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATAAAAGTCTTCGCACAGTGAAAACTAAAATGGA",
    "ATGCCTATTGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAG",
    "GAATGTTCCCAATAGTAGACATAAAAGTCTTCGCACAGTGAAAACTAAAATGGATCAAGCAGATGATGTTTCCTGTCCACTTCTAAATTCTTGTCTTAGT"
]

---

## Day 1: Getting started
You already have enough Python knowledge to start implementing our sequence assembly program. Let's start off by breaking down the problem into more managable parts (this is a skill you will develop very quickly while programming).

Step 1 says we need to compute [pairwise alignments](https://en.wikipedia.org/wiki/Sequence_alignment#Pairwise_alignment). To do this we need 2 things: a way to score alignments, and a way to generate alignments. An easy way to score an alignment is known as the [_edit distance_](https://en.wikipedia.org/wiki/Levenshtein_distance), which is simply the minimum number of changes (character insertions, deletions, or substitutions) that are required to transform one string into another.

### Edit distance: an example
Imagine I want to find the edit distance between the strings "kitten" and "sitting":

1. **k**itten → **s**itten (substitution of "s" for "k")
2. sitt**e**n → sitt**i**n (substitution of "i" for "e")
3. sittin → sittin**g** (insertion of "g" at the end).

So the _edit distance_ between "kitten" and "sitting" is 3.

Your first task is to write the beginnings of a function to compute the edit distance. Instead of a whole string, your input will be 2 characters. Please fill in the template provided for you and ensure it passes the test cases...

In [7]:
def edit_distance(queryA, queryB):
    if _:
        return _
    
    return _
    
assert edit_distance('A', 'T') == 1
assert edit_distance('G', 'G') == 0

AssertionError: 

Don't worry, you will soon extend this to longer sequences but it's a good start for now.

Another task we will need to do from step 3 of our algorithm is "merge" fragments. Your next task is to write a merging function that takes 2 fragments and merges them end-to-end in order. You can fill in the template below and ensure it passes the test cases...

In [8]:
def merge(fragA, fragB):
    return _
    
assert merge("", "ATG") == "ATG"
assert merge("ATG", "") == "ATG"
assert merge("ATG", "CCT") == "ATGCCT"
assert merge("A", "TG") == "ATG"

AssertionError: 

Well done! You have now completed the beginnings of the sequence assembly program. It doesn't look like much now but it's a great foundation for tomorrow. Congratulate yourself, you've earned it!

---

## Day 2: Sequence assembly made easy!
![How to draw an owl](https://i.kym-cdn.com/photos/images/newsfeed/000/572/078/d6d.jpg)

Now that you can iterate over sequences, it's time to complete your _edit distance_ function to work on sequences longer than a single character. Please fill out the function below and ensure it passes the test cases:

In [9]:
def edit_distance(queryA, queryB):
    ...
    
assert edit_distance('A', 'T') == 1
assert edit_distance('G', 'G') == 0
assert edit_distance('kitten', 'sitting') == 3
assert edit_distance('', '') == 0
assert edit_distance('ABCD', 'EFGH') == 4
assert edit_distance('ABCD', 'ZBCZ') == 2

AssertionError: 

Great work! Now it's time to generate alignments :)

In [10]:
from project import assemble, needleman_wunsch

assemble(fragments, needleman_wunsch)

len(fragments)=12
-TGCCTATTGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGA
ATGCCTATTGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAG-
99
len(fragments)=11
------GGCTGACTCTGCCGCTGTACCAATCTCCTGTAAAAGAATTAGATAAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATAAAAGTCTTCG
AGCAAGGGCTGACTCTGCCGCTGTACCAATCTCCTGTAAAAGAATTAGATAAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATAAAAG------
94
len(fragments)=10
-----------CAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGAACTTTCTTCAGAAGCTCCACCCTA
CAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGAACTTTCTTCAGAA-----------
89
len(fragments)=9
ATGCCTATTGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGA------------------------
--------------CAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGAACTTTCTTCAGAAGCTCCACCCTA
87
len(fragments)=8
------

['ATGCCTATTGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCAGATTTAGGACCAATAAGTCTTAATTGGTTTGAAGAACTTTCTTCAGAAGCTCCACCCTATAATTCTGAACCTGCAGAAGAATCTGAACATAAAAACAACAATTACGAACCAAACCTATTTAAAACTCCACAAAGGAAACCATCTTATAATCAGCTGGCTTCAACTCCAATAATATTCAAAGAGCAAGGGCTGACTCTGCCGCTGTACCAATCTCCTGTAAAAGAATTAGATAAATTCAAATTAGACTTAGGAAGGAATGTTCCCAATAGTAGACATAAAAGTCTTCGCACAGTGAAAACTAAAATGGATCAAGCAGATGATGTTTCCTGTCCACTTCTAAATTCTTGTCTTAGT']